# TP1 
## Estruturas Criptográficas - Criptografia e Segurança da Informação

### Exercício 2

2. Use o “package” Cryptography para
    1. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto [+Capítulo 1: Primitivas Criptográficas Básicas](https://paper.dropbox.com/doc/Capitulo-1-Primitivas-Criptograficas-Basicas-YAcE9VWuF88R2fmPyvKlx#:uid=971079522289346670472132&h2=AEAD-com-%E2%80%9CTweakable-Block-Ciph).  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
    2. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

##### Instalar packages necessários

In [ ]:
%pip install cryptography

##### Resumo 
Divisão em duas partes: 

1. a implementação da AEAD com Tweakable Block Ciphers usando o pacote "Cryptography"
2. construção do canal privado de informação assíncrona com acordo de chaves usando "X448 key exchange" e "Ed448 Signing&Verification".

##### Parte 1: Implementação da AEAD com Tweakable Block Ciphers

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import secrets


def tweakable_block_cipher(key, tweak):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    return encryptor.update(tweak) + encryptor.finalize()


def generate_aes_key():
    # Gera uma chave AES de 256 bits (32 bytes)
    return secrets.token_bytes(32)

def generate_tweak():
    # Gera um tweak de 128 bits (16 bytes)
    return secrets.token_bytes(16)

# Exemplo de uso
aes_key = generate_aes_key()
tweak = generate_tweak()

print("AES Key:", aes_key)
print("Tweak:", tweak)

# Exemplo de uso
#key = b'your_aes_key'  # Substitua com sua chave AES
#tweak = b'your_tweak'  # Substitua com seu tweak
result = tweakable_block_cipher(aes_key, tweak)
print(result)


AES Key: b'`\xf3\xc6\r\x86F\x1ecU\x9f\xb0\x00c\x1a\x08\x8f|\xff\x06\xb7\xc9\xb6f\x02\xff\x0f\xbd\xa2\x11$kR'
Tweak: b'\xd0\xb2\xb4C\xd5\xc805U\x924G\xc4\x8b\x99\xa0'
b'\xd3\x96\x04\x822\xce\xd4\xf58I\xb9\r\x89\xb6\x13i'


##### Parte 2: Construção do Canal Privado Assíncrono

In [3]:
from cryptography.hazmat.primitives.asymmetric import x448, ed448
from cryptography.hazmat.backends import default_backend

# Função para gerar chaves X448
def generate_x448_key():
    private_key = x448.X448PrivateKey.generate()
    public_key = private_key.public_key()
    return private_key, public_key

# Função para gerar chaves Ed448
def generate_ed448_key():
    private_key = ed448.Ed448PrivateKey.generate()
    public_key = private_key.public_key()
    return private_key, public_key

# Função para realizar o acordo de chaves X448
def x448_key_exchange(my_private_key, their_public_key):
    shared_key = my_private_key.exchange(their_public_key)
    return shared_key

# Função para assinar uma mensagem usando Ed448
def sign_message(message, private_key):
    signature = private_key.sign(message)
    return signature

# Função para verificar a assinatura usando Ed448
def verify_signature(message, signature, public_key):
    try:
        public_key.verify(signature, message)
        return True
    except Exception:
        return False

# Exemplo de uso
# Alice
alice_private_key_x448, alice_public_key_x448 = generate_x448_key()
alice_private_key_ed448, alice_public_key_ed448 = generate_ed448_key()

# Bob
bob_private_key_x448, bob_public_key_x448 = generate_x448_key()
bob_private_key_ed448, bob_public_key_ed448 = generate_ed448_key()

# Acordo de chaves X448
alice_shared_key = x448_key_exchange(alice_private_key_x448, bob_public_key_x448)
bob_shared_key = x448_key_exchange(bob_private_key_x448, alice_public_key_x448)

# Confirmação da chave acordada
if alice_shared_key == bob_shared_key:
    print("Chave acordada com sucesso!")

# Assinatura e verificação usando Ed448
message = b'Mensagem a ser assinada e verificada'
alice_signature = sign_message(message, alice_private_key_ed448)
verification_result = verify_signature(message, alice_signature, alice_public_key_ed448)

if verification_result:
    print("Assinatura verificada com sucesso!")
else:
    print("Falha na verificação da assinatura.")


Chave acordada com sucesso!
Assinatura verificada com sucesso!


# NOTAS FINAIS : 

MUDAR ISTO PARA CYTHON

Multiprocessing

Ver trabalhos anos anteriores

Fazer testes

Explicar melhor os packages e kernel do miniconda neste arquivo